---
#### 작업형 1-1. 출동시간과 도착시간 차이가 평균적으로 가장 오래 걸린 소방서의 시간을 분으로 변환해 출력하라
---

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-1-1.csv")
print(df.shape)
#display(df.head(2))

df['출동시간'] = pd.to_datetime(df['출동시간'])
df['도착시간'] = pd.to_datetime(df['도착시간'])
df['경과시간(분)'] = (df['도착시간'] - df['출동시간']).dt.total_seconds() / 60

tmp = df.groupby('소방서')['경과시간(분)'].mean().reset_index()
ans = tmp.sort_values('경과시간(분)', ascending=False).iloc[0,-1]
print('정답 = ', round(ans))

(100, 3)
정답 =  81


---
#### 작업형 1-2. 학교에서 교사 1인당 담당 학생수가 가장 많은 학교를 찾고, 그 학교 전체 교사수를 구하라
---

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-1-2.csv")
print(df.shape)
#display(df.head(2))

df['1인당학생수'] = (df['1학년']+ df['2학년']+ df['3학년']+ df['4학년']+ df['5학년']+ df['6학년']) / df['교사수']
ans = df.sort_values('1인당학생수', ascending = False)
print('정답 = ', ans.iloc[0,1])

(100, 8)
정답 =  19


---
#### 작업형 1-3. 연도별 총 범죄건수(범죄유형별 발생건수의 합)의 연도별 평균 값을 구한 후 그 값이 가장 큰 연도를 찾아 해당연도의 총 범죄건수의 월평균 값을 구하라 (반올림 정수 출력)
---

In [53]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-1-3.csv")
print(df.shape)
#display(df.head())

#df['총범죄건수'] = df.iloc[:, 1:7].sum(axis=1)
df['총범죄건수'] = df['강력범죄']+df['절도범죄']+df['폭력범죄']+df['지능범죄']+df['풍속범죄']+df['교통범죄']
df['연도'] = df['날짜'].str[:4]
df['월'] = df['날짜'].str[6:8]

tmp = df.groupby('연도')['총범죄건수'].mean().reset_index()
tmp.sort_values('총범죄건수', ascending=False) ## 2023년

df_2023 = df[df['연도'] == '2023']
print('정답 = ', round(df_2023['총범죄건수'].mean()))

(60, 8)
정답 =  533


---
#### 작업형 2-1. 건물데이터기반으로 건물의 난방부하(에너지 효율성)을 예측하라
- 타겟 : Heat_Load (Very Low -> Mdeium -> Very High)
- 평가 : F1 score macro
- 제출 : result.csv (pred)
---

In [74]:
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_test.csv")
print(train.shape, test.shape)
display(train.head(2), test.head(2))

############ EDA ############################################################
# 숫자형, 범주형 변수 혼재. 결측치 없음
# train/test obj변수들의 카테고리 종류 동일함
#############################################################################

# 1.타겟분리
y = train['Heat_Load']
train = train.drop('Heat_Load', axis=1)

# 2.수치형변수 스케일링
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = StandardScaler()
num_cols = ['Compac', 'Surf_Area', 'Wall_Area', 'Glaze_Area', 'Glaze_Distr', 'Cool_Load']
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

# 3.범주형변수 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 4.검증데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=42)

# 5. model, eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
model = RandomForestClassifier(random_state=120)
model.fit(X_train, y_train)

model_pred = model.predict(X_valid)
score = f1_score(y_valid, model_pred, average='macro')
print('f1 score(macro) = ', score)

#submit
pred = model.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv('result.csv', index=False)

(537, 10) (231, 9)


,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load,Heat_Load
0,0.74,686.0,245.0,220.5,Short,South,0.25,3,14.72,Very Low
1,0.98,514.5,294.0,Small,Tall,South,0.40,2,33.94,High


,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load
0,0.64,784.0,343.0,220.5,Short,South,0.4,4,22.25
1,0.82,612.5,318.5,Large,Tall,North,0.4,3,32.43


f1 score(macro) =  0.9458791402396054


---
#### 작업형 3-1. 감기약 위약(플라시보)를 투여받은 환자의 부작용은 항암약 위약을 투여받은 환자의 부작용과 차이가 있는가?

- 감기약(위약)을 투여받은 환자들의 부작용 발생비율
| 구분 | 부작용  | 비율 |
|-----|-------|-----|
| 1   | 아픔   | 10% |
| 2   | 조금아픔|5%   |
| 3   | 속쓰림 | 15%  |
| 4   | 이상 없음| 70% |
---

In [85]:
# 항암약(위약)을 투여받은 환자들의 부작용
df = pd.DataFrame({
    "항암약":[4,4,3,4,1,4,1,4,1,4,4,2,1,4,2,3,2,4,4,4]
    })
display(len(df), df.head())

## 카이제곱 검정을 위해, observed와 expected 구하기
expected = [20*0.1, 20*0.05, 20*0.15, 20*0.7]
observed = df['항암약'].value_counts().sort_index().to_list()
print(expected, observed)

## 카이제곱 검정
from scipy.stats import chisquare, chi2_contingency
result = chisquare(observed, expected)
print(result)
print('정답 :', '두 약의 부작용은 동일한 분포를 보인다.' )

20

,항암약
0,4
1,4
2,3
3,4
4,1


[2.0, 1.0, 3.0, 14.0] [4, 3, 2, 11]
Power_divergenceResult(statistic=np.float64(6.976190476190476), pvalue=np.float64(0.07266054733847573))
정답 : 두 약의 부작용은 동일한 분포를 보인다.


---
#### 작업형 3-2. 다중선형 회귀모델을 구축하고, 각 소문제에 답하라
- 종속변수: temperature / 독립변수: solar, wind, O3
1. 주어진 데이터를 기반으로 다중선형 회귀모델을 구축하고, 독립변수 O3의 회귀계수를 구하라
2. 위 모델에서 solar와 O3이 고정된 상태에서 wind의 세기가 증가함에 따라 temperature가 감소함. wind의 회귀계수에 대한 p값을 구하라
3. 위 모델에서 solar=100, wind=5, o3=30일때의 예측값을 구하라
---

In [86]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-3-2.csv")
print(df.shape)
display(df.head())

(100, 4)


,solar,wind,o3,temperature
0,89.14,6.28,33.52,23.0
1,109.97,1.04,27.01,20.7
2,102.83,6.42,41.00,20.5
3,84.94,10.20,33.44,22.2
4,94.21,4.95,29.97,21.4


In [87]:
from statsmodels.formula.api import ols, logit
model = ols('temperature ~ solar + wind + o3', data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            temperature   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.464
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.229
Time:                        10:51:04   Log-Likelihood:                -195.45
No. Observations:                 100   AIC:                             398.9
Df Residuals:                      96   BIC:                             409.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0507      1.994      9.555      0.000      15.093      23.008
solar          0.0039      0.015      0.251      0.802      -0.027       0.035
wind          -0.0252      0.090     -0.280      0.780      -0.204       0.153
o3             0.0749      0.036      2.079      0.040       0.003       0.146
==============================================================================
Omnibus:                        0.654   Durbin-Watson:                   2.328
Prob(Omnibus):                  0.721   Jarque-Bera (JB):                0.672
Skew:                           0.187   Prob(JB):                        0.715
Kurtosis:                       2.855   Cond. No.                     1.20e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [90]:
# 1. O3의 회귀계수 = 0.0749
# 2. wind의 회귀계수 p값 = 0.780

# 3. solar=100, wind=5, o3=30일때의 예측값 : 21.56163
new_data = pd.DataFrame({
    'solar':[100],
    'wind':[5],
    'o3':[30]
})
pred = model.predict(new_data)
pred

,0
0,21.56163
